# Tool Basics

In [2]:
import rich
from llama_stack_client import Agent, LlamaStackClient, AgentEventLogger
from llama_stack_client.lib.agents.client_tool import client_tool

In [3]:
@client_tool
def get_location(query: str = "location"):
    """
    Provide the location upon request.

    :param query: The query from user
    :returns: Information about user location
    """
    import geocoder
    
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

In [4]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [5]:
client.models.list()

[Model(identifier='meta-llama/Llama-3.2-3B-Instruct', metadata={}, api_model_type='llm', provider_id='ollama', provider_resource_id='llama3.2:3b-instruct-fp16', type='model', model_type='llm'),
 Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', provider_resource_id='all-minilm:latest', type='model', model_type='embedding')]

In [6]:
client.toolgroups.list()

[ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', provider_resource_id='builtin::websearch', type='tool_group', args=None, mcp_endpoint=None),
 ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', provider_resource_id='builtin::rag', type='tool_group', args=None, mcp_endpoint=None)]

In [7]:
client.tools.list()

[Tool(description='Search the web for information', identifier='web_search', parameters=[Parameter(description='The query to search for', name='query', parameter_type='string', required=True, default=None)], provider_id='tavily-search', provider_resource_id='web_search', tool_host='distribution', toolgroup_id='builtin::websearch', type='tool', metadata=None),
 Tool(description='Insert documents into memory', identifier='insert_into_memory', parameters=[], provider_id='rag-runtime', provider_resource_id='insert_into_memory', tool_host='distribution', toolgroup_id='builtin::rag', type='tool', metadata=None),
 Tool(description='Search for information in a database.', identifier='knowledge_search', parameters=[Parameter(description='The query to search for. Can be a natural language sentence or keywords.', name='query', parameter_type='string', required=True, default=None)], provider_id='rag-runtime', provider_resource_id='knowledge_search', tool_host='distribution', toolgroup_id='builtin:

In [8]:
agent = Agent(
    client, 
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant.",
    tools=[get_location],
)


In [9]:
user_prompts = [
    "Where am I?",
]
session_id = agent.create_session("test-session")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")

    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=True,
    )
    for log in AgentEventLogger().log(response):
        log.print()

User> Where am I?

inference> [get_location(query="location")]


/Users/diego/src/ai-masterclass-2025/.venv/lib/python3.12/site-packages/geocoder/uscensus.py:36: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('^\d+', self.address, re.UNICODE)


tool_execution> Tool:get_location Args:{'query': 'location'}
tool_execution> Tool:get_location Response:"Your current location is: Toronto, Ontario, CA"
inference> It seems like you're in Toronto! Would you like to know more about this city or perhaps get directions somewhere else?
